d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Aggregations, JOINs and Nested Queries
Apache Spark&trade; and Databricks&reg; allow you to create on-the-fly data lakes.

## In this lesson you:
* Use basic aggregations.
* Correlate two data sets with a join
* Use subselects

## Audience
* Primary Audience: Data Analysts
* Additional Audiences: Data Engineers and Data Scientists

## Prerequisites
* Web browser: **Chrome**
* A cluster configured with **8 cores** and **DBR 6.3**
* Familiarity with <a href="https://www.w3schools.com/sql/" target="_blank">ANSI SQL</a> is required

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Setup & Classroom-Cleanup<br>

For each lesson to execute correctly, please make sure to run the **`Classroom-Setup`** cell at the start of each lesson (see the next cell) and the **`Classroom-Cleanup`** cell at the end of each lesson.

In [4]:
%run "./Includes/Classroom-Setup"

<iframe  
src="//fast.wistia.net/embed/iframe/b9v2h8520r?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/b9v2h8520r?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

-sandbox
## Basic aggregations

Using <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions" target="_blank">built-in Spark functions</a>, you can aggregate data in various ways. 

Run the cell below to compute the average of all salaries in the `People10M` table.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> By default, you get a floating point value.

<iframe  
src="//fast.wistia.net/embed/iframe/1g8ch2l14y?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/1g8ch2l14y?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [8]:
%sql
SELECT avg(salary) AS averageSalary 
FROM People10M

averageSalary
72633.0076033


Convert that value to an integer using the SQL `round()` function. See
<a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.round" class="text-info">the PySpark documentation for <tt>round()</tt></a>
for more details.

In [10]:
%sql
SELECT round(avg(salary)) AS averageSalary 
FROM People10M

averageSalary
72633.0


In addition to the average salary, what are the maximum and minimum salaries?

In [12]:
%sql
SELECT max(salary) AS max, min(salary) AS min, round(avg(salary)) AS average 
FROM People10M

max,min,average
180841,-26884,72633.0


## Joining two tables

Correlate the data in two data sets using a SQL join. 

The `People10M` table has 10 million names in it. 

> How many of the first names appear in Social Security data files? 

To find out, use the `SSANames` table with first name popularity data from the United States Social Security Administration. 

For every year from 1880 to 2014, `SSANames` lists the first names of people born in that year, their gender, and the total number of people given that name. 

By joining the `People10M` table with `SSANames`, weed out the names that aren't represented in the Social Security data.

(In a real application, you might use a join like this to filter out bad data.)

<iframe  
src="//fast.wistia.net/embed/iframe/9quo6ugich?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/9quo6ugich?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

Start by taking a quick peek at what `SSANames` looks like.

In [16]:
%sql
SELECT * FROM SSANames limit 5

firstName,gender,total,year
Jennifer,F,54336,1983
Jessica,F,45278,1983
Amanda,F,33752,1983
Ashley,F,33292,1983
Sarah,F,27228,1983


Next, get an idea of how many distinct names there are in each of our tables, with a quick count of distinct names.

In [18]:
%sql
SELECT count(DISTINCT firstName) 
FROM People10M

count(DISTINCT firstName)
5113


In [19]:
%sql
SELECT count(DISTINCT firstName)
FROM SSANames

count(DISTINCT firstName)
93889


By introducing two more temporary views, each one consisting of distinct names, the join will be easier to read/write.

In [21]:
%sql
CREATE OR REPLACE TEMPORARY VIEW SSADistinctNames AS 
  SELECT DISTINCT firstName AS ssaFirstName 
  FROM SSANames;

CREATE OR REPLACE TEMPORARY VIEW PeopleDistinctNames AS 
  SELECT DISTINCT firstName 
  FROM People10M

Next, join the two tables together to get the answer.

In [23]:
%sql
SELECT firstName 
FROM PeopleDistinctNames 
INNER JOIN SSADistinctNames ON firstName = ssaFirstName 
limit 5

firstName
Alayna
Melaine
Faye
Tegan
Lashanda


How many are there?

In [25]:
%sql
SELECT count(*) 
FROM PeopleDistinctNames 
INNER JOIN SSADistinctNames ON firstName = ssaFirstName

count(1)
5096


## Nested Queries

Joins are not the only way to solve the problem. 

A sub-select works as well.

<iframe  
src="//fast.wistia.net/embed/iframe/hs7vn1o0et?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/hs7vn1o0et?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

In [28]:
%sql
SELECT count(firstName) 
FROM PeopleDistinctNames
WHERE firstName IN (
  SELECT ssaFirstName FROM SSADistinctNames
)

count(firstName)
5096


-sandbox
## Exercise 1

In the tables above, some of the salaries in the `People10M` table are negative. 

These salaries represent bad data. 

Your job is to convert all the negative salaries to positive ones, and then sort the top 20 people by their salary.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** See the Apache Spark documentation, <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions" target="_blank">built-in functions</a>.

### Step 1
Create a temporary view called `PeopleWithFixedSalaries`, where all the negative salaries have been converted to positive numbers.

In [31]:
%sql
CREATE OR REPLACE TEMPORARY VIEW PeopleWithFixedSalaries AS
  select id, firstname, middleName, lastName, gender, birthDate, ssn,
    CASE 
      WHEN salary < 0 THEN abs(salary)
      ELSE salary
    END as salary
  from People10M 


In [32]:
# TEST - Run this cell to test your solution.

belowZero = spark.read.table("PeopleWithFixedSalaries").where("salary < 0").count()
dbTest("SQL-L3-belowZero", 0, belowZero)

print("Tests passed!")

Tests passed!

### Step 2

Starting with the table `PeopleWithFixedSalaries`, create another view called `PeopleWithFixedSalariesSorted` where:
0. The data set has been reduced to the first 20 records
0. The records are sorted by the column `salary` in ascending order

In [34]:
%sql
CREATE OR REPLACE TEMPORARY VIEW PeopleWithFixedSalariesSorted AS 
  select * 
  from PeopleWithFixedSalaries
  order by salary asc 
  limit 20
  

In [35]:
# TEST - Run this cell to test your solution.

resultsDF = spark.sql("select salary from PeopleWithFixedSalariesSorted")
dbTest("SQL-L3-count", 20, resultsDF.count())

results = [r[0] for r in resultsDF.collect()]

dbTest("SQL-L3-fixedSalaries-0", 2, results[0])
dbTest("SQL-L3-fixedSalaries-1", 3, results[1])
dbTest("SQL-L3-fixedSalaries-2", 4, results[2])

dbTest("SQL-L3-fixedSalaries-10", 19, results[10])
dbTest("SQL-L3-fixedSalaries-11", 19, results[11])
dbTest("SQL-L3-fixedSalaries-12", 20, results[12])

dbTest("SQL-L3-fixedSalaries-17", 28, results[17])
dbTest("SQL-L3-fixedSalaries-18", 30, results[18])
dbTest("SQL-L3-fixedSalaries-19", 31, results[19])

print("Tests passed!")

Tests passed!

## Exercise 2

As a refinement, assume that all salaries under $20,000 represent bad rows and filter them out.

Additionally, categorize each person's salary into $10K groups.

### Step 1
Create a temporary view called `PeopleWithFixedSalaries20K` where:
0. Start with the table `PeopleWithFixedSalaries`
0. The data set excludes all records where salaries are below $20K
0. The data set includes a new column called `salary10k`, that should be the salary in groups of 10,000. For example:
  * A salary of 23,000 should report a value of "2"
  * A salary of 57,400 should report a value of "6"
  * A salary of 1,231,375 should report a value of "123"

In [38]:
%sql
CREATE OR REPLACE TEMPORARY VIEW PeopleWithFixedSalaries20K AS
  select * , round(salary/10000) as salary10k
  from PeopleWithFixedSalaries
  where salary > 20000


In [39]:
%sql
select salary10k, count(*) as total from PeopleWithFixedSalaries20K  group by salary10k order by salary10k, total limit 5

salary10k,total
2.0,43784
3.0,212630
4.0,536536
5.0,1055261
6.0,1623248


In [40]:
# TEST - Run this cell to test your solution.

below2K = spark.sql("select * from PeopleWithFixedSalaries20K where salary < 20000").count()
dbTest("SQL-L3-count-salaries", 0, below2K)

resultsDF = spark.sql("select salary10k, count(*) as total from PeopleWithFixedSalaries20K  group by salary10k order by salary10k, total limit 5")
results = [ (str(int(r[0]))+" w/"+str(r[1])) for r in resultsDF.collect()]

dbTest("SQL-L3-countSalaries-0", "2 w/43784", results[0])
dbTest("SQL-L3-countSalaries-1", "3 w/212630", results[1])
dbTest("SQL-L3-countSalaries-2", "4 w/536536", results[2])
dbTest("SQL-L3-countSalaries-3", "5 w/1055261", results[3])
dbTest("SQL-L3-countSalaries-4", "6 w/1623248", results[4])

print("Tests passed!")

Tests passed!

## Exercise 3

Using the `People10M` table, count the number of females named Caren who were born before March 1980.

### Step 1
Starting with the table `People10M`, create a temporary view called `Carens` where:
0. The result set has a single record
0. The data set has a single column named `total`
0. The result counts only 
  * Females (`gender`)
  * First Name is "Caren" (`firstName`)
  * Born before March 1980 (`birthDate`)

In [43]:
%sql
create or replace temporary view Carens as 
  select count(*) as total
  from People10M
  where gender = 'F' AND firstName = 'Caren' AND year(birthDate) < 1980



In [44]:
# TEST - Run this cell to test your solution.

rows = spark.sql("SELECT total FROM Carens").collect()
dbTest("SQL-L3-carens-len", 1, len(rows))
dbTest("SQL-L3-carens-total", 750, rows[0].total)

print("Tests passed!")

Tests passed!

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Classroom-Cleanup<br>

Run the **`Classroom-Cleanup`** cell below to remove any artifacts created by this lesson.

In [46]:
%run "./Includes/Classroom-Cleanup"

## Next Steps

* Do the extra [Challenge Exercise]($./Optional/SSQL 03L - Joins Aggregations Lab).
* Or start one of the following lessons:
  - [Accessing Data - Amazon S3]($./SSQL 04a - Accessing Data - Amazon S3).
  - [Accessing Data - Azure Blob]($./SSQL 04b - Accessing Data - Azure Blob).

## Additional Topics & Resources

* <a href="https://docs.databricks.com/spark/latest/spark-sql/index.html" target="_blank">Spark SQL Reference</a>
* <a href="http://spark.apache.org/docs/latest/sql-programming-guide.html" target="_blank">Spark SQL, DataFrames and Datasets Guide</a>
* <a href="https://databricks.com/blog/2017/08/31/cost-based-optimizer-in-apache-spark-2-2.html" target="_blank">Cost-based Optimizer in Apache Spark 2.2</a>

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>